In [1]:
import sys
import os
import time
import torch.nn as nn
from tqdm.notebook import tqdm

sys.path.append(os.path.abspath('..'))

from compiling_nn.build_odd import compile_nn

In [2]:
class Model(nn.Module):
    def __init__(self, input_size, *hidden_layers) -> None:
        hidden_layers = list(hidden_layers)
        layers = [nn.Linear(x, y) for x, y in zip([input_size]+hidden_layers, hidden_layers)]
        super().__init__()
        self.nn = nn.Sequential(*layers)

In [3]:
def input_size_exp(max_rep=1):
    results = dict()
    with tqdm(total=5*max_rep, desc="Input Size Experiment") as pbar:
        for in_size in [5,10,15,20,25]:
            res = 0
            for _ in range(max_rep):
                model = Model(in_size, 10, 1)
                start = time.process_time()
                compile_nn(model)
                res+= (time.process_time() - start)
                pbar.update(1)
            results[in_size] = res/max_rep
    return results

def hidden_size_exp(max_rep=1):
    results = dict()
    with tqdm(total=5*max_rep, desc="Hidden Layer Size Experiment") as pbar:
        for hl_size in [5,10,15,20,25]:
            res = 0
            for _ in range(max_rep):
                model = Model(15, hl_size, 1)
                start = time.process_time()
                compile_nn(model)
                res+= (time.process_time() - start)
                pbar.update(1)
            results[hl_size] = res/max_rep
    return results

def hidden_num_exp(max_rep=1, in_size=15, hl_size=5, nb_layer=5):
    results = dict()
    with tqdm(total=nb_layer*max_rep, desc="Hidden Layer Number Experiment") as pbar:
        for hl_num in range(1,1+nb_layer):
            res = 0
            for _ in range(max_rep):
                model = Model(in_size, *[hl_size]*hl_num, 1)
                start = time.process_time()
                compile_nn(model)
                res+= (time.process_time() - start)
                pbar.update(1)
            results[hl_num] = res/max_rep
    return results

In [4]:
nb_iter = 5
for _ in range(nb_iter):
    print(input_size_exp(1))
for _ in range(nb_iter):
    print(hidden_size_exp(1))
for _ in range(nb_iter):
    print(hidden_num_exp(1))

Input Size Experiment:   0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
print(hidden_num_exp(25, in_size=15, hl_size=10, nb_layer=10))

Hidden Layer Number Experiment:   0%|          | 0/250 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
print(hidden_num_exp(25, in_size=20))

Hidden Layer Number Experiment:   0%|          | 0/125 [00:00<?, ?it/s]

{1: 16.75581863175983, 2: 45.5593901035202, 3: 88.2789686694798, 4: 82.45634129416008, 5: 117.93491455776012}
